Prerequisites:

`./AppData/Local/Programs/Python/Python310/Scripts/pip install plotly nbformat`

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
import plotly.express as px

In [2]:
path = r"C:/Users/sulta/git/cone-operator-lab/data/eigenvalues/box_a1.0_b1.5_c2.3_N20000.txt"
eigs_all = np.loadtxt(path, ndmin=1)
eigs_all = np.sort(eigs_all[eigs_all > 0.0])

In [3]:
n_steps = [2000, 5000, 10000]
factors = np.arange(10, 120, 2)
m_grid = 151

A2_true = (1.0 + 1.5 + 2.3) / (4 * np.pi**2)

In [4]:
rows = []
for n in n_steps:
    eigs = eigs_all[:n]
    lamN = eigs[-1]
    t_max = 500 / lamN  # 1 = λN/λN

    for f in factors:
        t_min = f / lamN
        if t_min >= t_max:  # invalid window
            continue

        # log-spaced t grid
        t = np.geomspace(t_min, t_max, m_grid)

        # Z(t) = sum_k exp(-λ_k t)  (vectorized)
        Z = np.exp(-np.outer(eigs, t)).sum(axis=0)

        # Design matrix: [t^-3/2, t^-1, t^-1/2, 1]
        X = np.column_stack([t**(-1.5), t**(-1.0), t**(-0.5), np.ones_like(t)])

        # least squares (stable, uses LAPACK)
        coef, *_ = lstsq(X, Z)

        A2_hat = (coef[2] * (4*np.pi)**(1.5)) / (4*np.pi**3)
        err_pct = abs(A2_hat / A2_true - 1.0) * 100.0

        rows.append((n, f, err_pct))

df = pd.DataFrame(rows, columns=["N", "f", "err_pct"])

In [5]:
fig = px.line(
    df, x="f", y="err_pct", color="N",
    log_y=True,
    labels={"f": "tMin factor (f/λN)", "err_pct": "relative error of A2 (%)"},
    title="A2 reconstruction error vs tMin factor"
)
fig.update_layout(
    width=900,
    height=550,
    margin=dict(l=80, r=40, t=80, b=70)
)
fig.update_traces(mode="lines+markers", hovertemplate="N=%{legendgroup}<br>f=%{x}<br>err=%{y:.4g}%<extra></extra>")
fig.show()